In [ ]:
# ====== 0. 필요 라이브러리 다운로드 ======
print("모든 필수 라이브러리를 최신 버전으로 설치")
!pip install -U transformers accelerate peft trl datasets pandas rich
!pip install -U bitsandbytes

# ====== 1. 라이브러리 임포트 & 환경 설정======
print("\n 라이브러리 불러오기")
import os
import pandas as pd
import numpy as np
import torch
from datasets import Dataset
from transformers import DataCollatorForLanguageModeling, BitsAndBytesConfig
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
)
from trl import SFTTrainer, SFTConfig

from tqdm.auto import tqdm
tqdm.pandas()
progress_callbacks = []  # 빈 리스트로 대체


# ====== 2.환경 변수 설정 ======
os.environ["WANDB_DISABLED"] = "true"  # W&B 비활성화
BASE = "/content/drive/MyDrive/DILAB/MARS/mimic-iv-note_2.2/files/note/input_output"

TRAIN_INPUT   = f"{BASE}/discharge_train_input.parquet"
TRAIN_OUTPUT  = f"{BASE}/discharge_train_output.parquet"
TEST_INPUT    = f"{BASE}/discharge_test_input.parquet"
TEST_OUTPUT   = f"{BASE}/discharge_test_output.parquet"

# Qwen 모델 경로
MODEL_DIR = "/content/drive/MyDrive/DILAB/qwen3-8b"
OUTPUT_DIR = "/content/drive/MyDrive/DILAB/qwen3-8b/logs/original_1"
# 학습 결과 QLoRA 어댑터를 저장할 경로
ADAPTER_DIR = "/content/drive/MyDrive/DILAB/qwen3-8b/adapter/original_1"

MAX_SEQ_LENGTH = 25_000        # 시퀀스 길이(토큰) 목표
TRAIN_LIMIT    = 10_000        # 학습 샘플 수 제한 (None: 전체 사용)
VALID_LIMIT    = 2_000         # 검증 샘플 수 제한 (원하면 조절/None)
SHUFFLE_SEED   = 42            # 재현성용 시드
PACKING        = False         # TRL packing (짧은 샘플 합치기). 긴 샘플 위주면 False 권장.
EPOCH = 1                      # 학습 에폭 수

KEYS = ["subject_id", "hadm_id", "note_id"] # 각 데이터셋을 불러올 기준은 KEY 목록

# ====== 3. 구글 드라이브 마운트 =====
from google.colab import drive
drive.mount('/content/drive')

# ====== 4. Train/Test 파일에서 랜덤으로 TRAIN_LIMIT만큼 데이터 행 개수 꺼내오기 ======
# --- 4-1. 데이터 로드하기 ---
train_input_df = pd.read_parquet(TRAIN_INPUT)
train_output_df = pd.read_parquet(TRAIN_OUTPUT)

print(f"✅ TRAIN INPUT rows: {len(train_input_df):,}") # :, 포맷 지정자=> 숫자를 천 단위 구분 쉼표로 표시함
print(f"✅ TRAIN OUTPUT rows: {len(train_output_df):,}")

# --- 4-2. KEYS 기준으로 input, output 파일을 inner join (두 테이블의 컬럼들이 합쳐짐) ---
train_merged_df = train_input_df.merge(train_output_df, on=KEYS, how="inner")
  # -> on: 어떤 KEY들을 기준으로 join할 것인지
  # -> how: 어떤 방식으로 join할 것인지
print(f"✅ TRAIN MERGED rows: {len(train_merged_df):,}")

# --- 4-3. 병합한 데이터들 중, BHC나 DI가 비어있다면 해당 데이터는 제거함 ---
both_empty = (train_merged_df["bhc_text"].fillna("").str.len() == 0) & \
             (train_merged_df["di_text"].fillna("").str.len() == 0)
  # -> fillna(""): NaN 값을 빈 문자열로 대체하는 코드
  # -> .str.len(): 문자열로 변환한 뒤 해당 문자열의 길이를 반환
train_merged_df = train_merged_df[~both_empty].reset_index(drop=True)
  # -> ~both_empty: both_empty에는 각 행마다 True/False가 있는데, "~" 비트 Not 연산자로 값을 뒤집는다
  # -> reset_index(drop=True) : 데이터 프레임의 인덱스를 0부터 순서대로 다시 매겨주는 함수
    # drop=True 옵션을 주면, 기존 인덱스를 새로운 컬럼으로 남기지 않고 완전히 버린다.

# --- 4-4. 필터링된 데이터셋에서 TRAIN_LIMIT만큼 랜덤시드로 데이터 꺼내오기 ---
# TRAIN_LIMIT가 None이 아니고, 전체 데이터보다 작을 때만 꺼내오기
if TRAIN_LIMIT is not None and TRAIN_LIMIT < len(train_merged_df):
    sample_df = train_merged_df.sample(n=TRAIN_LIMIT, random_state=SHUFFLE_SEED).reset_index(drop=True)
    # .sample(): 난수 생성기를 사용해서 데이터프레임의 행 중 일부를 무작위로 선택함.
    # reset_index(drop=True) : 무작위로 섞이면서 기존 인덱스가 섞였기에, 0부터 순서대로 인덱스를 다시 매겨준다.
else:
  # TRAIN_LIMIT보다 전체 데이터가 작은 경우에는 해당 데이터 copy해서 그대로 사용하기
    sample_df = train_merged_df.copy()

print(f"🎯 최종 선택된 샘플 수: {len(sample_df):,}")


# ====== 5. 프롬프트 포맷팅: train_prompts 리스트에 각 학습용 Chat 형식으로 변환 ======
# --- 5-1. 예시 프롬프트 ---
EXAMPLES = r"""
Brief Hospital Course
- Admitted with progressive abdominal distension and confusion in the setting of cirrhosis.
- Diagnostic paracentesis negative for SBP; CXR unremarkable; UA/cultures without growth.
- Initiated diuretics: Furosemide 40 mg PO daily; Spironolactone 50 mg PO daily (dose chosen given K+ 4.5).
- Symptomatic improvement with diuresis; electrolytes stable on regimen.
- Outpatient follow-up arranged for liver clinic and endoscopic screening as noted in input.

Discharge Instructions
- Take Furosemide 40 mg PO daily and Spironolactone 50 mg PO daily exactly as prescribed.
- Follow a low-sodium diet as instructed in the input.
- Attend the scheduled outpatient visits listed in the input.
- Return for care if you develop abdominal pain, fever, or confusion as specified in the input.
"""

INSTRUCTION = (
    """
You are an expert physician-writer who crafts hospital discharge summaries.

OBJECTIVE
- Produce the best possible “Brief Hospital Course” (BHC) and “Discharge Instructions” (DI) for this admission.
- Be accurate, concise, and high-yield.

EVIDENCE RULES
- Use ONLY facts explicitly present in the input for this admission.
- If something is not stated, omit it (do not guess or add placeholders).
- Preserve all numbers/units/doses/medication names exactly as written when cited.
- Expand abbreviations on first mention (e.g., “glomerular basement membrane (GBM)”).

STYLE
- English, clinical, crisp, and readable.
- Prefer bullets for scannability; short paragraphs are acceptable.
- De-duplicate; avoid narrative fluff.
- Maintain **chronologic clarity** (e.g., “On POD #1...”, “By discharge...” when timing is available).
- Sensible ordering: presentation → key findings → diagnostic workup → treatments/changes (with stated rationale) → **daily or staged progression if described** → response/status at discharge → pending studies/follow-up items.

OUTPUT FORMAT (exactly two top-level headings; no others)
Brief Hospital Course
- Summarize the course of this admission only.
- Include: presenting problem; salient exam/imaging/labs (with values only if meaningful); pivotal decisions and treatments (med changes with stated reasons); **postoperative or inpatient progression by day/stage if mentioned**; clinical response/trends; discharge status; and explicit pending items for follow-up.
- Explicitly mention removal or initiation of tubes, lines, diet advancement, or ambulation if these are provided in input.

Discharge Instructions
- Patient-facing, concise, and strictly derived from input directions.
- Include only actionable items explicitly present in the input, such as:
  • medication directions or changes (dose/formulation/frequency as written),
  • ordered labs/procedures and when to obtain them (if specified),
  • **activity restrictions (lifting, driving, exercise) and wound/incision care** if provided,
  • monitoring tasks (e.g., glucose logs) and diet/activity instructions if stated,
  • whom/where to follow up if named in the input.
- Do NOT restate the entire medication list unless it is explicitly provided as patient instructions.
- Do NOT add generic or invented advice (e.g., hydration, smoking cessation) unless the input specifies it.

GUARDRAILS
- No new clinicians, appointments, or tests beyond the input.
- Do not invent demographics/PHI (age, phone, MRN, addresses).
- Do not label values as high/low/improved unless the input does so (e.g., *, H, L, or explicit wording).
- Do NOT repeat any disease/problem header (e.g., “# GERD”, “# Hypertension”, “# Anemia”) more than once. If the same problem appears multiple times in the input, **summarize it once** under a single header and merge relevant details.
- Each header beginning with “#” must be unique. Duplicate section headers are not allowed.
- If multiple sections have identical titles or nearly identical wording, combine them into one concise entry.

FINALIZATION (silent self-check before output; do not print the checklist)
- Exactly two headings are present and correctly titled.
- Every sentence is supported by the input.
- All doses/units/numbers match the input verbatim when cited.
- BHC covers presentation, key findings, treatments/changes with rationale, **time-based or staged progression**, and discharge condition.
- DI contains only actionable, patient-facing directions (medications, wound care, activity, follow-up) explicitly derived from the input.
    """
    f"{EXAMPLES}\n"
)

# --- 5-3. 학습용 Chat 형식으로 프롬프트 변환하기 ---
train_prompts = []

print("🪄 Chat-format로 변환 중...")

# 데이터 프레임의 iterrows() 함수로 for문 순회하여, 각 row 꺼내오기
for _, row in tqdm(sample_df.iterrows(), total=len(sample_df), desc="Formatting", ncols=100):
            # ㄴ> tqdm이 총 길이를 알고 있어서 정확한 % 표시 가능
            # ㄴ> 설명, 출력 길이 조절
  # system 메시지 (지시문)
  system_msg = {"role": "system", "content": INSTRUCTION}

  # user: input 텍스트(학습 데이터)
  user_msg = {"role": "user", "content": row["text"].strip()}

  # assistant: BHC + DI 결합한 데이터
  assistant_text = ""
  if row["bhc_text"].strip():
      assistant_text += f"Brief Hospital Course\n{row['bhc_text'].strip()}\n\n"
      # strip(): 앞뒤로 공백 제거하는 함수
  if row["di_text"].strip():
      assistant_text += f"Discharge Instructions\n{row['di_text'].strip()}"
  assistant_msg = {"role": "assistant", "content": assistant_text.strip()}

  # messages 구조로 위 3가지 묶은 후 train_prompts 리스트에 추가하기
  train_prompts.append([
      system_msg,
      user_msg,
      assistant_msg
  ])
print(f"✅ Chat-format 변환 완료! 총 {len(train_prompts):,}개 샘플")

# <<예시>>
# train_prompts = [
#   [
#     {"role": "system", "content": "You are a medical summarization model..."},
#     {"role": "user", "content": "환자는 흉통으로 내원하여 ..."},
#     {"role": "assistant", "content": "Brief Hospital Course\n환자는 안정 및 약물치료 후 호전됨.\n\nDischarge Instructions\n퇴원 후 외래 방문 예정."}
#   ],
#   [
#     {"role": "system", "content": "You are a medical summarization model..."},
#     {"role": "user", "content": "입원 중 발열로 검사 시행 ..."},
#     {"role": "assistant", "content": "Brief Hospital Course\n감염 증상 호전됨.\n\nDischarge Instructions\n항생제 복용 후 외래 추적."}
#   ]
# ]


# ====== 6. 학습 환경 설정 ======
# --- 6-1. 4비트 양자화 설정 ---
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)
  # BitsAndBytesConfig : 트랜스포머의 클래스 중 하나로, 양자화 시키는데 사용됨
  # load_in_4bit=True : 선형 레이어의 nf4 OR FP4로 대체하여 4비트 양자화 설정한다는 의미
  # bnb_4bit_quant_type="nf4" : 선형 레이어를 nf4로 대체하여 4비트 양자화 설정
    # [nf4 vs fp4의 차이점]
      # => 둘 다 4비트 양자화 방식이지만, FP4는 단순한 4비트 부동소수점 표현임.
      # => NF4는 데이터 분포(정규분포)에 맞춘 최적화된 4비트 표현이라서 학습 안정성, 정확도가 더 좋음
  # bnb_4bit_compute_dtype=torch.bfloat16 : 연산 방법을 bf16으로 함.
    # [bfloat16 연산]
      # => 16비트 부동소수점 포맷. fp32와 동일한 지수 범위를 가지면서 가수(정밀도)는 줄여서 메모리 절약 & 속도 향상을 동시에 얻음
      # => bf16은 정밀도는 떨어지지만 32비트의 너비를 갖고 있음. (넓은 수의 표현이 가능함)
    # FP32는 가장 정확하지만 무겁고, FP16은 가볍지만 범위가 좁아 불안정할 수 있음
    # bfloat16은 그 중간 지점에서 안정성 & 효율성을 챙길 수 있음

# --- 6-2. 모델 & 토크나이저 로드 ---
model = AutoModelForCausalLM.from_pretrained(
    MODEL_DIR,
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config,
    device_map="auto",
    offload_buffers=True
)
  # torch_dtype=torch.bfloat16 : 사용할 dtype 지정
  # quantization_config=bnb_config : QLoRA 적용
  # device_map="auto" : 모델을 메모리에 올릴 때 사용할 방식

model.config.use_cache = False # 추론 시, 이전 토큰의 attention 계산을 저장해서 속도를 높이는 설정. 학습할 때는 해당 캐시가 역전파를 방해함

# model.gradient_checkpointing_enable()
  # -> GPU 메모리를 절약하기 위한 설정
  # -> 원래는 각 레이어의 중간 activation 값을 GPU에 계속 저장해야 gradient를 계산할 수 있다.
  # -> 해당 설정을 키면 forward 시에 일부 중간 결과를 버리고, backward 과정에서 다시 계산하게 된다,
  # 따라서 메모리는 절약되지만 속도는 약간 느려진다.

model = prepare_model_for_kbit_training(model)
  # [prepare_model_for_kbit_training()]
    # -> BitsAndBytes 훈련용 모델을 LoRA와 호환되게 세팅해주는 함수

tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR, trust_remote_code=True)
  # -> trust_remote_code : 모델 저장소에 커스텀 토크나이저 코드가 있으면, 신뢰하고 불러오겠다는 의미
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
  # -> 패딩을 오른쪽에 붙이도록 설정
  # 토큰화 후 길이가 다른 문장들을 batch로 묶을 때, 짧은 문장의 뒤쪽에 [PAD] 토큰을 붙인다.
tokenizer.max_seq_length = MAX_SEQ_LENGTH
  # -> 최대 시퀀스 길이 설정



# --- 6-3. apply_chat_template으로 채팅 형식 프롬프트 하나의 문자열로 직렬화 하기 ---
# 직렬화한 text를 Dataset의 "text" 필드에 넣는다.
messages = [
    tokenizer.apply_chat_template(
        msgs,
        tokenize=False,             # 변환된 텍스트를 토큰화하지 않고 문자열 그대로 반환 (True로 하면 토큰 id 리스트를 반환함)
        add_generation_prompt=False # 마지막에 "Assistant:" 같은 답변 시작 프롬프트 붙일지 여부 지정
) for msgs in train_prompts
]
  # -> 토크나이저에 정의된 chat template을 적용해서, 모델이 이해할 수 있는 입력 문자열로 변환됨

print("📁 직렬화된 데모 텍스트 문자열:")
print(messages[0])


# --- 6-4. 직렬화된 텍스트 리스트로 Dataset 생성하기 (SFTTrainer에 넣어주기 위해) ---
train_ds = Dataset.from_dict({"messages":messages})


# --- 6-5. LoRA 설정 & 모델에 적용하기 ---
peft_config = LoraConfig(
    r=64,                                         # LoRA 랭크. 데이터셋이 복잡할수록 높은 값이 필요함.
    lora_alpha=16,                                # LoRA의 scailing factor. 낮으면 기존 데이터를 위주로, 높으면 새로운 데이터 위주로 받아들이게 하는 요소. 8, 16을 주로 사용함
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,                            # LoRA의 dropout 확률 설정
    bias="none",                                  # 편향
    task_type="CAUSAL_LM"                         # 작업 타입
)
model = get_peft_model(model, peft_config)
  # -> 기존 model에 peft_config를 적용해서 PEFT 모델 객체로 변환함
  # -> PEFT : 매개변수 효율적 미세조정
model.print_trainable_parameters()
  # -> 모델의 전체 파라미터 수 & 학습 가능한 파라미터 수를 출력함




# --- 6-6. 훈련 하이퍼 파라미터 설정 ---
sft_args = SFTConfig(
    per_device_train_batch_size=2,    # 각 GPU당 배치 사이즈(한번에 처리하는 샘플 개수. 한 스텝에서 n개의 샘플을 처리함), 기본값은 8이다
    gradient_accumulation_steps=1,    # 기울기 축적 단계, 기본 값은 1이며, 커질수록 정확도가 커지며, 배치 사이즈가 클 수록 메모리가 커지는 현상을 해결하기 위해 사용
                                      # 기울기를 여러 스텝 동안 모았다가 한번에 업데이트 ((배치크기 * 해당 변수값) 마다 1번의 배치가 수행됨.)

    gradient_checkpointing=True,      # 기본값은 False. True로 설정하면 역방향 시 메모리를 절약함
    max_grad_norm=0.3,                # 최대 기울기 표준. 기본값은 1.0. 그레디언트 클리핑을 위한 변수
    num_train_epochs=EPOCH,           # 훈련할 에폭 수
    learning_rate=2e-4,               # 학습률
    bf16=True,                        # 32비트 훈련 대신 bf16 16비트 혼합된 정밀도 훈련 사용할지 여부
    save_total_limit=3,               # 해당 속성을 설정하면, 오래된 체크포인트 순으로 삭제한다.(3인 경우, 3개난 가장 최신화된 3개이면서 과거의 것은 삭제하며 저장함)
                                      # 모델을 훈련하다 보면, output_dir에 checkpoint-1000 이런식으로 중간 저장본이 계속 쌓임
                                      # 이때, 이게 너무 많이 쌓이면 용량이 폭발하므로 save_total_limit는 가장 최근 M개만 남기고 나머지는 자동 삭제함

    logging_steps=10,                 # 로깅하는 단위. n번마다 로깅됨
    output_dir=OUTPUT_DIR,            # 결과 경로
    optim="paged_adamw_32bit",        # 사용하려는 옵티마이저
    lr_scheduler_type="cosine",       # 학습률이 시간에 따라 변화는 방식을 지정함(cosine 형태로 부드럽게 감소함)
                                      # 사용하는 스케줄러 타입. 기본값은 "linear". 종류는 consine 등이 있음

    warmup_ratio=0.05,                # 웜업 비율(초반에 학습률을 0->목표 학습률까지 천천히 올리는 비율). 기본값은 0. 0에서 learning_rate까지 선형 준비 단계에서 사용되는 총 훈련 단계의 비율
                                      # 훈련 초기에 너무 빠르게 학습률을 올리면 모델이 gradient explosion할 수 있음.
                                      # -> 전체 학습 스텝 중 5% 동안은 lr이 0 -> 2e-4까지 점진적으로 증가함

    max_steps=-1,                     # 수행할 총 훈련 단계 수. -1로 지정하면 기본값으로 자동으로 설정됨
    report_to='tensorboard',           # 훈련 중 기록되는 로그, 지표를 어디로 보낼지 결정하는 옵션
                                      # TensorBoard 로그로 기록함.(Colab에서 시각화 가능함)
    packing=False,                      # 여러 개의 짧은 샘플을 하나의 긴 시퀀스로 이어붙여(max_seq_length에 맞게) 학습하는 기능
    max_length=MAX_SEQ_LENGTH,         # SFTConfig에서 max_length 속성으로 지정해주어야 함!!!!!!!!
    completion_only_loss=True
)

# --- 6-7. SFTTrainer 설정 ---
# train_ds를 tokenized 상태로 변환하기
def tokenize_function(example):
    return tokenizer(
        example["messages"],
        truncation=True,
        max_length=MAX_SEQ_LENGTH,
        padding=False,
    )
tokenized_ds = train_ds.map(tokenize_function, batched=True)

trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_ds,     # 토큰화된 학습용 데이터셋 지정
    args=sft_args,                  # 훈련 하이퍼 파라미터 지정
)

print("tokenizer max_seq_length:" + str(tokenizer.max_seq_length))
print("trainer max_length:" + str(trainer.args.max_length))

print("🪄 학습 시작")
trainer.train()

# ====== 7. 어댑터 저장하기 ======
trainer.model.save_pretrained(ADAPTER_DIR)
tokenizer.save_pretrained(ADAPTER_DIR)
print(f"✅ LoRA 어댑터 저장 완료: {ADAPTER_DIR}")




모든 필수 라이브러리를 최신 버전으로 설치
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.3/506.3 kB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 136.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.4/243.4 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 60.4 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: rich
    Found existing installation: rich 13.9.4
    Uninstalling rich-13.9.4:
      Successfully uninstalled rich-13.9.4
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
  Attempting uninstall: datasets
   

Formatting:   0%|                                                         | 0/10000 [00:00<?, ?it/s]

✅ Chat-format 변환 완료! 총 10,000개 샘플


`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

...

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


tokenizer max_seq_length:25000
trainer max_length:25000
🪄 학습 시작


Step,Training Loss
10,2.362900
20,2.377700
30,2.323800
40,2.267400
50,2.181700
60,2.112000
70,2.086700
80,2.064900
90,1.973200
100,1.886600


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using

✅ LoRA 어댑터 저장 완료: /content/drive/MyDrive/DILAB/qwen3-8b/adapter/original_1


# Qwen3 Use

In [ ]:
# ====== 0. 필요 모델 import & 다운로드 ======
!pip install -U "transformers>=4.42.0" accelerate peft trl datasets pandas bitsandbytes rich tqdm

import os, re, pandas as pd, torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
from typing import List
from tqdm import tqdm
from google.colab import drive
drive.mount('/content/drive')


# ====== 1. 필요 변수 선언 ======
MAX_SEQ_LENGTH = 20000       # 모델 입력 최대 길이(상한). 너무 크게 잡으면 비효율적일 수 있음
GENERATION_MAX_NEW_TOKENS = 1024  # 새로 생성할 토큰 수(실사용 권장: 512~2048)
SHUFFLE_SEED = 13            # 랜덤 시드
TEST_LIMIT = 1               # 테스트할 데이터 개수
KEYS = ["subject_id", "hadm_id", "note_id"]

MODEL_DIR = "/content/drive/MyDrive/DILAB/qwen3-8b"  # base 모델 경로
ADAPTER_DIR = "/content/drive/MyDrive/DILAB/qwen3-8b/adapter/original_1"
BASE_DATA   = "/content/drive/MyDrive/DILAB/MARS/mimic-iv-note_2.2/files/note/input_output"

TEST_INPUT  = f"{BASE_DATA}/discharge_test_input.parquet"
TEST_OUTPUT = f"{BASE_DATA}/discharge_test_output.parquet"
TRAIN_INPUT = f"{BASE_DATA}/discharge_train_input.parquet"
TRAIN_OUTPUT= f"{BASE_DATA}/discharge_train_output.parquet"


# ====== 2. 모델 로드하기 ======
# --- 2-1. 4비트 양자화 ---
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

device_map = {"": 0} if torch.cuda.is_available() else "cpu"

# --- 2-2. original_model & trained_model 모델 로드 ---
original_model = AutoModelForCausalLM.from_pretrained(
    MODEL_DIR,
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    max_length=MAX_SEQ_LENGTH,
    device_map=device_map,
    offload_buffers=True
)

base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_DIR,
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    max_length=MAX_SEQ_LENGTH,
    device_map=device_map,
    offload_buffers=True
)

trained_model = PeftModel.from_pretrained(
    base_model,
    ADAPTER_DIR,
    is_trainable=False  # 추론용
)

# --- 2-3. 모델 평가모드 전환 ---
trained_model.eval()

# ---- 2-4. 토크나이저 로드 ---
tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
tokenizer.max_seq_length = MAX_SEQ_LENGTH

print("\n✅ Base Model & Trained Model & 토크나이저 로드 완료!")


# ====== 3. 랜덤으로 TEST 데이터 뽑기 ======
# --- 3-1. 데이터 로드하기 ---
test_input_df = pd.read_parquet(TEST_INPUT)
test_output_df = pd.read_parquet(TEST_OUTPUT)

print(f"✅ TEST INPUT rows: {len(test_input_df):,}")
print(f"✅ TEST OUTPUT rows: {len(test_output_df):,}")

# --- 3-2. KEYS 기준으로 inner join ---
test_merged_df = test_input_df.merge(test_output_df, on=KEYS, how="inner")
print(f"✅ TEST MERGED rows: {len(test_merged_df):,}")

# --- 3-3. BHC와 DI가 모두 비어있는 행 제거 ---
both_empty = (test_merged_df["bhc_text"].fillna("").str.len() == 0) & \
             (test_merged_df["di_text"].fillna("").str.len() == 0)
test_merged_df = test_merged_df[~both_empty].reset_index(drop=True)

# --- 3-4. 샘플 추출 ---
if TEST_LIMIT is not None and TEST_LIMIT < len(test_merged_df):
    sample_df = test_merged_df.sample(n=TEST_LIMIT, random_state=SHUFFLE_SEED).reset_index(drop=True)
else:
    # 🔧 버그 수정: 정의되지 않은 train_merged_df 대신 test_merged_df 사용
    sample_df = test_merged_df.copy()

print(f"🎯 최종 선택된 테스트 샘플 수: {len(sample_df):,}")


# ====== 4. 프롬프트 설정하기 ======
INSTRUCTION = (
    """
You are an expert physician-writer who crafts hospital discharge summaries.

OBJECTIVE
- Produce the best possible “Brief Hospital Course” (BHC) and “Discharge Instructions” (DI) for this admission.
- Be accurate, concise, and high-yield.

EVIDENCE RULES
- Use ONLY facts explicitly present in the input for this admission.
- If something is not stated, omit it (do not guess or add placeholders).
- Preserve all numbers/units/doses/medication names exactly as written when cited.
- Expand abbreviations on first mention (e.g., “glomerular basement membrane (GBM)”).

STYLE
- English, clinical, crisp, and readable.
- Prefer bullets for scannability; short paragraphs are acceptable.
- De-duplicate; avoid narrative fluff.
- Maintain **chronologic clarity**.
- Sensible ordering: presentation → key findings → diagnostic workup → treatments/changes (with rationale) → progression if described → response/status at discharge → pending items.

OUTPUT FORMAT (exactly two top-level headings; no others)
Brief Hospital Course
- Summarize the course of this admission only.
- Include presentation; salient findings; pivotal decisions; meds/changes with reasons; progression; status at discharge; pending items; tubes/lines/diet/ambulation if provided.

Discharge Instructions
- Patient-facing, concise, strictly derived from input directions only (med directions, labs/procedures timing, activity restrictions, wound care, monitoring tasks, follow-up if named).
- Do NOT restate the full med list unless provided as instructions.
- No invented advice.

GUARDRAILS
- No new clinicians/tests beyond input.
- No invented PHI.
- Do not label values as high/low unless stated.
- Do NOT repeat a problem header more than once; merge duplicates.

FINALIZATION (silent self-check)
- Exactly two headings.
- Every sentence supported by input.
- All numbers/units match input.
- BHC has time-based clarity, DI only actionable directions explicitly in input.
    """
)

# --- 4-2. TEST용 데이터들 & 지시문을 Chat 형식으로 프롬프트 변환 ---
test_prompts = []
outputs = []
print("🪄 Chat-format로 변환 중...")

for _, row in tqdm(sample_df.iterrows(), total=len(sample_df), desc="Formatting", ncols=100):
    system_msg = {"role": "system", "content": INSTRUCTION}
    user_msg = {"role": "user", "content": row["text"].strip()}
    assistant_msg = {"role": "assistant", "content": ""}

    test_prompts.append([system_msg, user_msg, assistant_msg])

    outputs.append({"bhc": row["bhc_text"], "di": row["di_text"]})

print(f"✅ Chat-format 변환 완료! 총 {len(test_prompts):,}개 샘플")

# --- 4-3. chat 템플릿 직렬화 ---
print("\n🪄 chat 형식 프롬프트 -> 문자열로 직렬화 시작")
prompts = [
    tokenizer.apply_chat_template(
        prompt,
        tokenize=False,
        add_generation_prompt=True
    )
    for prompt in test_prompts
]
print("✅ 직렬화 완료!")
print("📁 직렬화된 데모 텍스트 문자열:")
print(prompts[0])


# ====== 5. 생성 유틸 ======
def _collect_eos_ids(tok):
    """모델이 사고 구간/대화 턴을 깔끔히 종료하도록 EOS 후보들을 수집합니다."""
    eos_list = []
    for sym in ["</think>", "<|im_end|>"]:
        tid = tok.convert_tokens_to_ids(sym)
        if tid is not None and tid != tok.unk_token_id:
            eos_list.append(tid)
    eos_list.append(tok.eos_token_id)
    # transformers >= 4.42에서 eos_token_id에 리스트 전달 가능
    return eos_list

def _strip_think(text: str) -> str:
    """출력에서 <think>...</think> 사고 구간을 제거합니다."""
    return re.sub(r"<think>.*?</think>", "", text, flags=re.DOTALL).strip()

def generate_model_output(model, prompt: str, do_sample: bool = True):
    # 5-1. 토크나이즈
    model_inputs = tokenizer(prompt, return_tensors="pt").to("cuda" if torch.cuda.is_available() else "cpu")

    # 5-2. 안전한 종료 지점 지정
    eos_ids = _collect_eos_ids(tokenizer)

    # 5-3. 생성
    generated = model.generate(
        model_inputs.input_ids,
        max_new_tokens=GENERATION_MAX_NEW_TOKENS,
        eos_token_id=eos_ids,
        pad_token_id=tokenizer.pad_token_id,
        do_sample=do_sample,
        top_p=0.9 if do_sample else None,
        temperature=0.7 if do_sample else None
    )

    # 5-4. 입력 부분 제거 -> 새로 생성된 부분만 디코딩
    gen_only = [out[len(inp):] for inp, out in zip(model_inputs.input_ids, generated)]
    text = tokenizer.batch_decode(gen_only, skip_special_tokens=True)[0]

    # 5-5. 사고 구간 숨김
    text = _strip_think(text)
    return text


# ====== 6. 각 프롬프트별 결과 출력 ======
i = 0
for prompt in prompts:
    print(f"\n\n-------------{i+1}번째 예시 ---------------")

    original_response = generate_model_output(original_model, prompt)
    trained_response = generate_model_output(trained_model, prompt)

    print("\n\n✅input:")
    print(prompt)

    print("\n\n✅original_model 출력 결과:")
    print(original_response)

    print("\n\n✅trained_model 출력 결과:")
    print(trained_response)

    print("\n\n📌output(정답):")
    print("\t📁bhc:")
    print(outputs[i]["bhc"])
    print("\t📁di:")
    print(outputs[i]["di"])
    i += 1


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]


✅ Base Model & Trained Model & 토크나이저 로드 완료!
✅ TEST INPUT rows: 68,589
✅ TEST OUTPUT rows: 68,589
✅ TEST MERGED rows: 68,589
🎯 최종 선택된 테스트 샘플 수: 1
🪄 Chat-format로 변환 중...


Formatting: 100%|███████████████████████████████████████████████████| 1/1 [00:00<00:00, 1459.40it/s]

✅ Chat-format 변환 완료! 총 1개 샘플

🪄 chat 형식 프롬프트 -> 문자열로 직렬화 시작
✅ 직렬화 완료!
📁 직렬화된 데모 텍스트 문자열:
<|im_start|>system

You are an expert physician-writer who crafts hospital discharge summaries.

OBJECTIVE
- Produce the best possible “Brief Hospital Course” (BHC) and “Discharge Instructions” (DI) for this admission.
- Be accurate, concise, and high-yield.

EVIDENCE RULES
- Use ONLY facts explicitly present in the input for this admission.
- If something is not stated, omit it (do not guess or add placeholders).
- Preserve all numbers/units/doses/medication names exactly as written when cited.
- Expand abbreviations on first mention (e.g., “glomerular basement membrane (GBM)”).

STYLE
- English, clinical, crisp, and readable.
- Prefer bullets for scannability; short paragraphs are acceptable.
- De-duplicate; avoid narrative fluff.
- Maintain **chronologic clarity**.
- Sensible ordering: presentation → key findings → diagnostic workup → treatments/changes (with rationale) → progression if describe



✅input:
<|im_start|>system

You are an expert physician-writer who crafts hospital discharge summaries.

OBJECTIVE
- Produce the best possible “Brief Hospital Course” (BHC) and “Discharge Instructions” (DI) for this admission.
- Be accurate, concise, and high-yield.

EVIDENCE RULES
- Use ONLY facts explicitly present in the input for this admission.
- If something is not stated, omit it (do not guess or add placeholders).
- Preserve all numbers/units/doses/medication names exactly as written when cited.
- Expand abbreviations on first mention (e.g., “glomerular basement membrane (GBM)”).

STYLE
- English, clinical, crisp, and readable.
- Prefer bullets for scannability; short paragraphs are acceptable.
- De-duplicate; avoid narrative fluff.
- Maintain **chronologic clarity**.
- Sensible ordering: presentation → key findings → diagnostic workup → treatments/changes (with rationale) → progression if described → response/status at discharge → pending items.

OUTPUT FORMAT (exactly two t

In [4]:
# ====== 0. 필요 모델 import & 다운로드 ======
!pip install -U "transformers>=4.42.0" accelerate peft trl datasets pandas bitsandbytes rich tqdm

import os, re, pandas as pd, torch
from typing import List, Tuple
from tqdm import tqdm
from google.colab import drive
drive.mount('/content/drive')

from transformers import (
    AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig,
    StoppingCriteria, StoppingCriteriaList, set_seed
)
from peft import PeftModel

# ====== 1. 필요 변수 선언 ======
MAX_SEQ_LENGTH = 20000              # 토크나이저에 줄 상한(긴 입력 대비)
CTX_TOKENS_LIMIT = 4096             # 실제 프롬프트 토큰 상한(모델 안정성)
GEN_MAX_NEW_TOKENS_BHC = 500        # BHC 생성 길이
GEN_MAX_NEW_TOKENS_DI  = 450        # DI 생성 길이
GEN_MIN_NEW_TOKENS     = 120        # 너무 일찍 끊김 방지
SHUFFLE_SEED = 13
TEST_LIMIT = 1
KEYS = ["subject_id", "hadm_id", "note_id"]

MODEL_DIR = "/content/drive/MyDrive/DILAB/qwen3-8b"  # base
ADAPTER_DIR = "/content/drive/MyDrive/DILAB/qwen3-8b/adapter/original_1"
BASE_DATA   = "/content/drive/MyDrive/DILAB/MARS/mimic-iv-note_2.2/files/note/input_output"

TEST_INPUT  = f"{BASE_DATA}/discharge_test_input.parquet"
TEST_OUTPUT = f"{BASE_DATA}/discharge_test_output.parquet"

set_seed(42)  # 재현성

# ====== 2. 모델 로드하기 ======
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
device_map = {"": 0} if torch.cuda.is_available() else "cpu"

original_model = AutoModelForCausalLM.from_pretrained(
    MODEL_DIR,
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    max_length=MAX_SEQ_LENGTH,
    device_map=device_map,
    offload_buffers=True
)
base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_DIR,
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    max_length=MAX_SEQ_LENGTH,
    device_map=device_map,
    offload_buffers=True
)
trained_model = PeftModel.from_pretrained(
    base_model,
    ADAPTER_DIR,
    is_trainable=False
)
trained_model.eval()

tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
tokenizer.max_seq_length = MAX_SEQ_LENGTH

print("\n✅ Base Model & Trained Model & 토크나이저 로드 완료!")

# ====== 3. 데이터 로드 & 샘플 ======
test_input_df = pd.read_parquet(TEST_INPUT)
test_output_df = pd.read_parquet(TEST_OUTPUT)
print(f"✅ TEST INPUT rows: {len(test_input_df):,}")
print(f"✅ TEST OUTPUT rows: {len(test_output_df):,}")

test_merged_df = test_input_df.merge(test_output_df, on=KEYS, how="inner")
print(f"✅ TEST MERGED rows: {len(test_merged_df):,}")

both_empty = (test_merged_df["bhc_text"].fillna("").str.len() == 0) & \
             (test_merged_df["di_text"].fillna("").str.len() == 0)
test_merged_df = test_merged_df[~both_empty].reset_index(drop=True)

if TEST_LIMIT is not None and TEST_LIMIT < len(test_merged_df):
    sample_df = test_merged_df.sample(n=TEST_LIMIT, random_state=SHUFFLE_SEED).reset_index(drop=True)
else:
    sample_df = test_merged_df.copy()
print(f"🎯 최종 선택된 테스트 샘플 수: {len(sample_df):,}")

# ====== 4. 공통 규칙/프롬프트 ======
INSTRUCTION = """You are an expert physician-writer who crafts hospital discharge summaries.

STRICT RULES:
- Use ONLY facts explicitly present in the INPUT (this admission only). If not stated, omit it. No inventions.
- Preserve numbers/units/doses/medication names verbatim when cited.
- Expand abbreviations on first mention.
- English, clinical, concise; prefer bullets; avoid fluff; chronological clarity.
- Mention tubes/lines/diet/ambulation ONLY if present in INPUT.
- No new clinicians/tests/appointments beyond INPUT.
- Do NOT label values as high/low/improved unless INPUT states it.
- Merge duplicates; avoid repeating a problem header more than once.
"""

BHC_TASK = """TASK:
Write only the section "Brief Hospital Course" as concise bullets or short paragraphs.
Cover: presentation → key findings → workup → treatments/changes with rationale → progression by day/stage if described → status at discharge → explicit pending items and follow-ups from INPUT. Include lines/tubes/diet/ambulation only if mentioned.
End your output with </END> (print it literally).
"""

DI_TASK = """TASK:
Write only the section "Discharge Instructions". Patient-facing, concise, and strictly derived from INPUT directions.
Include only actionable items explicitly present in INPUT (medication directions/changes, labs/procedures with timing, activity/wound care restrictions if stated, monitoring tasks, whom/where to follow up if named). Do NOT restate the full medication list unless provided as instructions.
Use bullets when possible. Avoid invented or generic advice. No extra sections.
End your output with </END> (print it literally).
"""

def build_prompt_bhc(input_text: str) -> str:
    return (
        f"<<INSTRUCTION>>\n{INSTRUCTION}\n"
        f"<<INPUT>>\n{input_text.strip()}\n"
        f"<<OUTPUT>>\nBrief Hospital Course\n{BHC_TASK}\n"
    )

def build_prompt_di(input_text: str, bhc_text_for_context: str) -> str:
    # DI 생성 시 BHC를 약간의 맥락으로 제공(입력 사실 반복 방지). 단, DI는 INPUT 사실만 허용됨을 재강조.
    return (
        f"<<INSTRUCTION>>\n{INSTRUCTION}\n"
        f"<<INPUT>>\n{input_text.strip()}\n"
        f"<<BHC (context only; do not invent beyond INPUT)>>\n{bhc_text_for_context.strip()}\n"
        f"<<OUTPUT>>\nDischarge Instructions\n{DI_TASK}\n"
    )

# ====== 5. 정지/후처리/트렁케이션 ======
class StopOnEndTag(StoppingCriteria):
    def __init__(self, tokenizer, stop_strings: List[str]):
        self.stop_ids = [tokenizer.encode(s, add_special_tokens=False) for s in stop_strings]

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        for s in self.stop_ids:
            L = len(s)
            if L and input_ids.shape[1] >= L:
                if torch.equal(input_ids[0, -L:], torch.tensor(s, device=input_ids.device)):
                    return True
        return False

def collect_eos_ids(tok):
    ids = []
    for sym in ["</think>", "<|im_end|>"]:
        tid = tok.convert_tokens_to_ids(sym)
        if tid is not None and tid != tok.unk_token_id:
            ids.append(tid)
    ids.append(tok.eos_token_id)
    return ids

def strip_think(text: str) -> str:
    return re.sub(r"<think>.*?</think>", "", text, flags=re.DOTALL).strip()

def dedup_consecutive_lines(text: str) -> str:
    lines = [l.rstrip() for l in text.splitlines()]
    out, prev = [], None
    for l in lines:
        if l != prev:
            out.append(l)
        prev = l
    return "\n".join(out).strip()

def numeric_whitelist_filter(text: str, input_text: str, enable=True) -> str:
    if not enable:
        return text
    nums = set(re.findall(r"[-+]?\d+(?:\.\d+)?", input_text))
    out_lines = []
    for line in text.splitlines():
        line_nums = set(re.findall(r"[-+]?\d+(?:\.\d+)?", line))
        if not line_nums or line_nums.issubset(nums):
            out_lines.append(line)
    return "\n".join(out_lines).strip()

def truncate_by_tokens(text: str, max_tokens: int) -> str:
    ids = tokenizer.encode(text, add_special_tokens=False)
    if len(ids) <= max_tokens:
        return text
    ids = ids[-max_tokens:]  # 뒤쪽(최근 정보) 보존
    return tokenizer.decode(ids, skip_special_tokens=False)

# ====== 6. 생성 유틸 (단계별) ======
GEN_KWARGS_COMMON = dict(
    do_sample=False,                 # greedy로 지시문 준수↑
    no_repeat_ngram_size=6,
    repetition_penalty=1.12,
)

def run_generate(model, prompt: str, max_new: int, min_new: int) -> str:
    # 입력 토큰 트렁케이션 (컨텍스트 안정성)
    prompt = truncate_by_tokens(prompt, CTX_TOKENS_LIMIT)

    eos_ids = collect_eos_ids(tokenizer)
    stopping = StoppingCriteriaList([StopOnEndTag(tokenizer, ["</END>"])])

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda" if torch.cuda.is_available() else "cpu")
    gen = model.generate(
        inputs.input_ids,
        max_new_tokens=max_new,
        min_new_tokens=min_new,
        eos_token_id=eos_ids,                # 사고/EOS 토큰에서 정지
        pad_token_id=tokenizer.pad_token_id,
        stopping_criteria=stopping,
        **GEN_KWARGS_COMMON
    )
    gen_only = [o[len(inputs.input_ids[0]):] for o in gen]
    text = tokenizer.decode(gen_only[0], skip_special_tokens=True)
    text = strip_think(text)
    text = text.replace("</END>", "").strip()
    text = dedup_consecutive_lines(text)
    return text

def generate_bhc(model, input_text: str) -> str:
    prompt = build_prompt_bhc(input_text)
    text = run_generate(model, prompt, max_new=GEN_MAX_NEW_TOKENS_BHC, min_new=GEN_MIN_NEW_TOKENS)
    text = numeric_whitelist_filter(text, input_text, enable=True)
    # 헤더 보장
    if not text.lower().startswith("brief hospital course"):
        text = "Brief Hospital Course\n" + text
    return text.strip()

def generate_di(model, input_text: str, bhc_text: str) -> str:
    prompt = build_prompt_di(input_text, bhc_text_for_context=bhc_text)
    text = run_generate(model, prompt, max_new=GEN_MAX_NEW_TOKENS_DI, min_new=GEN_MIN_NEW_TOKENS)
    text = numeric_whitelist_filter(text, input_text, enable=True)
    # 헤더 보장
    if not text.lower().startswith("discharge instructions"):
        text = "Discharge Instructions\n" + text
    return text.strip()

def generate_bhc_and_di(model, input_text: str) -> Tuple[str, str, str]:
    input_text_trunc = truncate_by_tokens(input_text, CTX_TOKENS_LIMIT)
    bhc = generate_bhc(model, input_text_trunc)
    di  = generate_di(model, input_text_trunc, bhc_text=bhc)
    final = f"{bhc}\n\n{di}"
    return bhc, di, final

# ====== 7. 실행 ======
for i, row in sample_df.iterrows():
    print(f"\n\n-------------{i+1}번째 예시 ---------------")
    input_text = row["text"]

    # 원본/학습 모델 각각 2-단계 생성 (BHC -> DI)
    o_bhc, o_di, original_response = generate_bhc_and_di(original_model, input_text)
    t_bhc, t_di, trained_response  = generate_bhc_and_di(trained_model,  input_text)

    print("\n\n✅input:")
    print(input_text)

    print("\n\n✅original_model 출력 결과:")
    print(original_response)

    print("\n\n✅trained_model 출력 결과:")
    print(trained_response)

    print("\n\n📌output(정답):")
    print("\t📁bhc:")
    print(row["bhc_text"])
    print("\t📁di:")
    print(row["di_text"])


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]


✅ Base Model & Trained Model & 토크나이저 로드 완료!
✅ TEST INPUT rows: 68,589
✅ TEST OUTPUT rows: 68,589
✅ TEST MERGED rows: 68,589
🎯 최종 선택된 테스트 샘플 수: 1


-------------1번째 예시 ---------------


✅input:
Name:  ___                  Unit No:   ___
 
Admission Date:  ___              Discharge Date:   ___
 
Date of Birth:  ___             Sex:   M
 
Service: SURGERY
 
Allergies: 
ACE Inhibitors / aspirin
 
Attending: ___
 
Chief Complaint:
fall
 
Major Surgical or Invasive Procedure:
none
 
History of Present Illness:
___ w/ PMH of HTN, HLD, and dementia presents as transfer from 
___ after a fall down stairs several days ago (in addition 
to falling out of the shower and hitting head several days prior 
to that). This AM he was unable to get out of bed so his wife 
had him go to ___. He denies any chest pain, shortness 
breath,
or dizziness. Says he has pain in his left side/flank when he 
moves but not at rest. CT head through pelvis showed rib 
fractures left ___ displaced, left ___ nondisplace